In [32]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [35]:
df = pd.read_csv("../output/loss.csv")
df.head()

,step,epoch,mesa,base
0,0,0,12.497458,10.910637
1,1,0,8.819035,7.821729
2,2,0,10.557375,7.866491
3,3,0,8.533783,8.117002
4,4,0,12.024865,10.225608


In [44]:
# Perform linear regression
X = sm.add_constant(df[loss1])  # Adds a constant term to the predictor
model = sm.OLS(df[loss2], X).fit()
predictions = model.get_prediction(X)
prediction_summary = predictions.summary_frame(alpha=0.05)  # 95% confidence interval

# Sort the data by loss1 for plotting
df_sorted = df.sort_values(by=loss1)
prediction_summary_sorted = prediction_summary.loc[df_sorted.index]

# Get regression coefficients and standard errors
coef = model.params
std_err = model.bse
intercept, slope = coef[0], coef[1]
intercept_err = std_err[0]
slope_err = std_err[1]

# Format the regression equation
regression_equation = f"y = ({slope:.2f}±{slope_err:.2f})x + ({intercept:.2f}±{intercept_err:.2f})"

# Create scatter plot
fig = go.Figure()

# Add scatter plot for actual data
fig.add_trace(go.Scatter(
    x=df[loss1],
    y=df[loss2],
    mode='markers',
    marker=dict(color='blue'),
    name='Data'
))

# Add regression line
fig.add_trace(go.Scatter(
    x=df_sorted[loss1],
    y=prediction_summary_sorted['mean'],
    mode='lines',
    line=dict(color='black'),
    name=f'Regression Line<br>{regression_equation}'
))

# Add confidence interval
fig.add_trace(go.Scatter(
    x=np.concatenate([df_sorted[loss1], df_sorted[loss1][::-1]]),
    y=np.concatenate([prediction_summary_sorted['mean_ci_upper'], prediction_summary_sorted['mean_ci_lower'][::-1]]),
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False,
    name='Confidence Interval'
))

# Update layout for fonts and axis scaling
fig.update_layout(
    title='Scatter Plot with Linear Regression and Confidence Interval',
    xaxis_title=loss1,
    yaxis_title=loss2,
    font=dict(
        family="Arial, sans-serif",
        size=14,
        color="black"
    ),
    xaxis=dict(
        range=[5, 30],  # Set the range for x-axis
        scaleanchor="y",
        scaleratio=1,
        constrain="domain"
    ),
    yaxis=dict(
        range=[5, 25],  # Set the range for y-axis
        scaleanchor="x",
        scaleratio=1,
        constrain="domain"
    ),
    autosize=False,
    width=600,  # Set the width of the plot
    height=600  # Set the height of the plot
)

# Show the plot
fig.show()

/tmp/ipykernel_2832869/1967130969.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_2832869/1967130969.py:15: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_2832869/1967130969.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [45]:
# Save the plot as a PDF
fig.write_image("../output/loss.pdf", format="pdf")